In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
(X_train,Y_train),(X_test,Y_test) = keras.datasets.cifar10.load_data()
X_train,X_test = X_train/255,X_test/255

In [3]:
class ComplexDense(keras.layers.Layer):
    
    def __init__(self, units, **kwargs):
        self.units = units
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        self.kernel_real = self.add_weight(
            shape=(input_shape[1], self.units),
            initializer='glorot_normal', name='kernel_real')
        self.kernel_imag = self.add_weight(
            shape=(input_shape[1], self.units),
            initializer='glorot_normal', name='kernel_imag')
        self.bias_real = self.add_weight(
            shape=(self.units,),
            initializer='zeros', name='bias_real')
        self.bias_imag = self.add_weight(
            shape=(self.units,),
            initializer='zeros', name='bias_imag')
        super().build(input_shape)
    
    def call(self, inputs):
        kernel = tf.complex(self.kernel_real * 0.1, self.kernel_imag * 0.1)
        bias = tf.complex(self.bias_real, self.bias_imag)
        forward = inputs @ kernel + bias
        forward = forward / tf.complex(K.clip(tf.abs(forward), 1., None), 0.)
        return forward
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0],self.units)

In [4]:
class ComplexConv2D(keras.layers.Layer):
    
    def __init__(self, channels, kernel_size, **kwargs):
        self.channels = channels
        self.kernel_size = kernel_size
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        self.kernel_real = self.add_weight(
            shape=(*self.kernel_size, input_shape[-1], self.channels),
            initializer='glorot_normal', name='kernel_real')
        self.kernel_imag = self.add_weight(
            shape=(*self.kernel_size, input_shape[-1], self.channels),
            initializer='glorot_normal', name='kernel_imag')
        self.bias_real = self.add_weight(
            shape=(self.channels,),
            initializer='zeros', name='bias_real')
        self.bias_imag = self.add_weight(
            shape=(self.channels,),
            initializer='zeros', name='bias_imag')
        super().build(input_shape)
    
    def call(self, inputs):
        a,b = tf.real(inputs), tf.imag(inputs)
        c,d = self.kernel_real * 0.1, self.kernel_imag * 0.1
        ac = tf.nn.conv2d(a, c, (1,1,1,1), 'SAME')
        bd = tf.nn.conv2d(b, d, (1,1,1,1), 'SAME')
        ad = tf.nn.conv2d(a, d, (1,1,1,1), 'SAME')
        bc = tf.nn.conv2d(b, c, (1,1,1,1), 'SAME')
        re = tf.nn.bias_add(ac - bd, self.bias_real)
        im = tf.nn.bias_add(ad + bc, self.bias_imag)
        forward = tf.complex(re, im)
        forward = forward / tf.complex(K.clip(tf.abs(forward), 1., None), 0.)
        return forward
    
    def compute_output_shape(self, input_shape):
        return (*input_shape[:3],self.channels)

In [5]:
class ComplexPooling2D(keras.layers.Layer):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        super().build(input_shape)
    
    def call(self, inputs):
        re = tf.nn.avg_pool(tf.real(inputs), (1,2,2,1), (1,2,2,1), 'VALID')
        im = tf.nn.avg_pool(tf.imag(inputs), (1,2,2,1), (1,2,2,1), 'VALID')
        forward = tf.complex(re, im)
        return forward
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1]//2, input_shape[2]//2, input_shape[3])

In [6]:
X = X_input = keras.layers.Input(X_train.shape[1:])
X = keras.layers.Lambda(lambda x: tf.complex(x, 0.))(X)
X = ComplexConv2D(16, (3,3))(X)
X = ComplexPooling2D()(X)
X = ComplexConv2D(32, (3,3))(X)
X = ComplexPooling2D()(X)
X = ComplexConv2D(64, (3,3))(X)
X = ComplexPooling2D()(X)
X = ComplexConv2D(128, (3,3))(X)
X = ComplexPooling2D()(X)
X = keras.layers.Flatten()(X)
X = ComplexDense(256)(X)
X = ComplexDense(128)(X)
X = ComplexDense(np.max(Y_train)+1)(X)
X = keras.layers.Lambda(lambda x: tf.abs(x))(X)
X = keras.layers.Softmax()(X)
M = keras.Model(X_input, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 32, 32, 3)         0         
_________________________________________________________________
complex_conv2d_1 (ComplexCon (None, 32, 32, 16)        896       
_________________________________________________________________
complex_pooling2d_1 (Complex (None, 16, 16, 16)        0         
_________________________________________________________________
complex_conv2d_2 (ComplexCon (None, 16, 16, 32)        9280      
_________________________________________________________________
complex_pooling2d_2 (Complex (None, 8, 8, 32)          0         
_________________________________________________________________
complex_conv2d_3 (ComplexCon (None, 8, 8, 64)          36992     
__________

In [7]:
M.fit(X_train, Y_train, validation_data=(X_test,Y_test), batch_size=64, epochs=5, callbacks=[
    keras.callbacks.ReduceLROnPlateau(patience=3, verbose=1)
])

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 118s 2ms/step - loss: 2.1927 - acc: 0.1763 - val_loss: 2.0920 - val_acc: 0.2743
Epoch 2/5
50000/50000 [==============================] - 115s 2ms/step - loss: 2.0565 - acc: 0.2769 - val_loss: 2.0343 - val_acc: 0.3035
Epoch 3/5
50000/50000 [==============================] - 115s 2ms/step - loss: 2.0100 - acc: 0.3193 - val_loss: 2.0044 - val_acc: 0.3236
Epoch 4/5
50000/50000 [==============================] - 115s 2ms/step - loss: 1.9732 - acc: 0.3577 - val_loss: 1.9609 - val_acc: 0.3886
Epoch 5/5
50000/50000 [==============================] - 115s 2ms/step - loss: 1.9477 - acc: 0.3874 - val_loss: 1.9422 - val_acc: 0.3975
